In [4]:
from hct.envs.low_level_env import LowLevelEnv

import functools
import jax
import os

from datetime import datetime
from jax import numpy as jp
import matplotlib.pyplot as plt

from IPython.display import HTML, clear_output

import brax

import flax
from brax import envs
from brax.io import model
from brax.io import json
from brax.io import html
from hct import train as ppo
from brax.training.agents.sac import train as sac
from absl import logging

import getpass
import socket
env = LowLevelEnv()


ModuleNotFoundError: No module named 'assets'

In [4]:
rng = jax.random.PRNGKey(seed=1)
jit_env_reset = jax.jit(env.reset)
state = jit_env_reset(rng=rng)
pipeline_state = state.pipeline_state
print((type(pipeline_state.q)))


<class 'jaxlib.xla_extension.ArrayImpl'>


In [32]:

pipeline_state = state.pipeline_state
'''print(pipeline_state.q)
print(pipeline_state.qd)
print(pipeline_state.contact)'''
"""
print(env.sys.link_types)
print(env.sys.link_names)
print(env.sys.q_size())
print(env.sys.qd_size())
print(env.sys.dof.limit[0])
print(env.sys.dof.limit[1])
print(env.sys.link_parents)"""

#print(env.sys.actuator.q_id)
'''

ja = 2'''
ja = jp.zeros(env.sys.num_links())

def ja_id(id):
    if env.sys.link_types[id] == '1':
        ja = 

q = pipeline_state.q
q_idx = jp.indices([env.sys.q_size()])
hinge_link_q_idx = env.sys.q_idx('1')
non_hinge_link_q_idx = jp.delete(q_idx ,hinge_link_q_idx)

def process_sequences(arr, bool_indices):
    # Create a boolean mask of the same length as the array
    mask = jp.zeros_like(arr, dtype=bool)
    
    # Set the mask to True at the given indices
    mask = mask.at[bool_indices].set(True)
    
    # Apply the mask to the array, replacing True sequences with 0
    output = jp.where(mask, 0, arr)
    
    # Remove the zeroed sequences (replaced True sequences) from the array
    output = output[output != 0]
    
    return output
print(q_idx)
print(non_hinge_link_q_idx)

[[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14]]
[0 1 2 3 4 5 6]


In [21]:

print(pipeline_state.x.pos)

[[ 0.08253369  0.00290565  0.5362767 ]
 [ 0.32979733  0.23447308  0.533688  ]
 [ 0.5957482   0.44432437  0.5350934 ]
 [-0.14708921  0.24757078  0.48956725]
 [-0.3566649   0.50960165  0.44279844]
 [-0.16472992 -0.22866178  0.5388654 ]
 [-0.41612804 -0.45574212  0.54060435]
 [ 0.3121566  -0.24175948  0.5829861 ]
 [ 0.54896325 -0.47949743  0.62959903]]


In [5]:


jit_env_reset = jax.jit(env.reset)
jit_env_step = jax.jit(env.step)
rollout = []

rng = jax.random.PRNGKey(seed=1)
state = jit_env_reset(rng=rng)
for _ in range(100):
  rollout.append(state.pipeline_state)
  act_rng, rng = jax.random.split(rng)
  a = jax.random.uniform(act_rng, shape=(8,), minval=-1,maxval=1)
  state = jit_env_step(state, a)

HTML(html.render(env.sys.replace(dt=env.dt), rollout))



KeyboardInterrupt: 

[ 1.8702186  -1.0460168   0.25144252  0.19435003  0.7827993  -0.5731185
 -0.1448742   0.49370173  0.509909    0.5419925  -0.7711398  -0.00300725
 -0.640114    0.03325571  0.69460624]
[ 0.29746193 -0.5805117   0.09030087 -0.1782912  -2.192063   -1.2315791
  1.9765209   0.38477463 -0.52946275 -2.2101529  -1.4984249  10.381127
 -1.2786419  -9.661179  ]
None
[ 7  8  9 10 11 12 13 14]
